In [29]:
import pandas as pd 
import joblib
import json


In [39]:
loaded_model = joblib.load('data/ml/random_forest.pkl')


In [40]:
df = pd.read_csv("data/ml/df_current.csv")
with open("data/ml/teams_dictionary.json", 'r') as f:
    teams_dictionary = json.load(f)

In [41]:
df["home_code"] = df.home.apply(lambda x: teams_dictionary.get(x, -1))
df["away_code"] = df.away.apply(lambda x: teams_dictionary.get(x, -1))

not_usefull_columns = [
    'season',
    'home',
    'away',
    'goals_home',
    'goals_away'    
    ]

data = df.drop(columns=not_usefull_columns)

In [42]:
data

,home_reserve_overall,home_subs_overall,home_titular_overall,home_attack_overall,home_defend_overall,away_reserve_overall,away_subs_overall,away_titular_overall,away_attack_overall,away_defend_overall,home_code,away_code
0,67.2,80.750000,84.818182,88.666667,84.142857,64.666667,77.666667,86.090909,84.000000,86.500000,16,15
1,67.2,80.750000,84.818182,88.666667,84.142857,61.300000,77.416667,82.090909,86.666667,80.000000,16,25
2,67.2,80.750000,84.818182,88.666667,84.142857,64.100000,80.166667,83.363636,84.333333,81.750000,16,6
3,67.2,80.750000,84.818182,88.666667,84.142857,63.700000,76.333333,86.727273,87.666667,86.833333,16,14
4,67.2,80.750000,84.818182,88.666667,84.142857,65.800000,76.500000,81.727273,83.000000,81.714286,16,13
...,...,...,...,...,...,...,...,...,...,...,...,...
375,63.5,69.666667,74.090909,75.000000,74.500000,59.700000,72.750000,75.090909,75.333333,72.800000,-1,21
376,63.5,69.666667,74.090909,75.000000,74.500000,62.600000,74.583333,75.818182,79.000000,75.500000,-1,18
377,63.5,69.666667,74.090909,75.000000,74.500000,61.000000,71.666667,75.909091,75.666667,75.250000,-1,3
378,63.5,69.666667,74.090909,75.000000,74.500000,62.285714,74.166667,73.636364,73.500000,72.166667,-1,26


In [43]:

predictions = loaded_model.predict(data)


In [44]:
df["forecast"] = loaded_model.predict(data) 

In [47]:
prediction_summary = df[[
    "home",
    "away",
    "forecast",
    ]]
teams = dict( zip( df.home.astype("category").cat.codes , df.home ) )


In [48]:
points_end_season = { team: 0 for team in teams.values()}

for index, row in prediction_summary.iterrows():
    if row.forecast > 1:
        points_end_season[row.home] += 3
        points_end_season[row.away] += 0
    elif  row.forecast < -1:
        points_end_season[row.home] += 0
        points_end_season[row.away] += 3
    else:
        points_end_season[row.home] += 1
        points_end_season[row.away] += 1

In [49]:
results = sorted( ((v,k) for k,v in points_end_season.items()), reverse=True)
for index, (points, team) in enumerate(results):
    print(f"{index + 1 }. {team} ({points}) ")


1. Manchester City (92) 
2. Liverpool (87) 
3. Manchester United (84) 
4. Chelsea (82) 
5. Tottenham Hotspur (70) 
6. Leicester City (56) 
7. Brentford (38) 
8. Arsenal (36) 
9. Newcastle United (34) 
10. Everton (33) 
11. West Ham United (32) 
12. Leeds United (29) 
13. Crystal Palace (29) 
14. Aston Villa (29) 
15. Wolverhampton Wanderers (28) 
16. Burnley (28) 
17. Southampton (27) 
18. Norwich City (27) 
19. Brighton & Hove Albion (27) 
20. Watford (24) 
